# <font color='blue'>Data Science Academy - Python Fundamentos - Capítulo 6</font>

## Download: http://github.com/dsacademybr

## Stream de Dados do Twitter com MongoDB, Pandas e Scikit Learn

## Preparando a Conexão com o Twitter

In [1]:
# Instala o pacote tweepy
!pip install tweepy

orange3 3.24.1 requires keyrings.alt, which is not installed.
orange3 3.24.1 requires python-louvain>=0.13, which is not installed.
orange3 3.24.1 requires serverfiles, which is not installed.
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
orange3 3.24.1 has requirement numpy>=1.16.0, but you'll have numpy 1.15.1 which is incompatible.
orange-canvas-core 0.1.11 has requirement pip>=18.0, but you'll have pip 10.0.1 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
# Importando os módulos Tweepy, Datetime e Json
from tweepy.streaming import StreamListener #Ouvir pelo twitter
from tweepy import OAuthHandler #Autenticação pelo twitter
from tweepy import Stream #Classe de tratamento de streaming do pacote tweepy
import time
from datetime import datetime
import json

Veja no manual em pdf como criar sua API no Twitter e configure as suas chaves abaixo.

In [3]:
# Adicione aqui sua Consumer Key
consumer_key = "ts1VSG249kwQZt3G4gfOYlT9w"

In [4]:
# Adicione aqui sua Consumer Secret 
consumer_secret = "c23ogyLe3kBCMAQ49IST9DEPCdaIirvCkP0WMBh18z4Uq1thzl"

In [5]:
# Adicione aqui seu Access Token
access_token = "1174700278043611141-S8PkoBl4bEtJyKetkLbtuPt9LU6N93"

In [6]:
# Adicione aqui seu Access Token Secret
access_token_secret = "TBmStUq5HrgXUcoH4gYbdVdE0YH9g5WYZLSBLZCIbzrkT"

In [7]:
# Criando as chaves de autenticação
#Criação do objeto de autenticação a partir da classe importada de autenticação 
auth = OAuthHandler(consumer_key, consumer_secret)

In [8]:
auth.set_access_token(access_token, access_token_secret)

In [9]:
# Criando uma classe para capturar os stream de dados do Twitter e 
# armazenar no MongoDB
class MyListener(StreamListener):   
    
    def __init__(self, time_limit = 60):
        self.start_time = time.time()
        self.limit = time_limit
        self.saveFile = open('streamings.json', 'a')
        super(StreamListener, self).__init__()
        
    def on_data(self, dados):
        if (time.time() - self.start_time) < self.limit:
#             abcd. = json.loads(dados) #Conversão para o formato json para gravar no 
#             #mongodb
            streamings = json.loads(dados)
            id_ = streamings["id"]
            created_at = streamings["created_at"]
            id_str = streamings["id_str"]
            text = streamings["text"]
            obj = {"id":id_,"created_at":created_at,"id_str":id_str,"text":text,}
            tweetind = col.insert_one(obj).inserted_id
            print (obj)
            self.saveFile.write(dados)
            self.saveFile.write('\n')
            return True
        else:
            self.saveFile.close()
            return False

In [10]:
# Criando o objeto mylistener
mylistener = MyListener()

In [13]:
# Criando o objeto mystream
#Isntanciando o objeto mystream
mystream = Stream(auth, listener = MyListener(time_limit = 20))
#mystream = Stream(minhas credenciais, meu objeto para ouvir os tweets)

## Preparando a Conexão com o MongoDB

In [14]:
# Importando do PyMongo o módulo MongoClient
from pymongo import MongoClient   

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
# Criando a conexão ao MongoDB
client = MongoClient('localhost', 27017)

In [ ]:
# Criando o banco de dados twitterdb
db = client.twitterdb

In [ ]:
# Criando a collection "col"
col = db.tweets 

In [ ]:
# Criando uma lista de palavras chave para buscar nos Tweets
keywords = ['Data Science']

## Coletando os Tweets

In [ ]:
# Iniciando o filtro e gravando os tweets no MongoDB
# mystream.tweepy.Stream(auth = api.auth, listener = StreamListener(time_limit=20))
mystream.filter(track=keywords)

## --> Pressione o botão Stop na barra de ferramentas para encerrar a captura dos Tweets

## Consultando os Dados no MongoDB

In [23]:
mystream.disconnect()

In [19]:
# Verificando um documento no collection
col.find_one()

{'_id': ObjectId('5de27752075f4da22e0a62ab'),
 'id': 1200778024431562759,
 'created_at': 'Sat Nov 30 14:06:05 +0000 2019',
 'id_str': '1200778024431562759',
 'text': "RT @Neenar568: OK...it's a toss up~~I'm dying!!!! 🤣😂😂🤣 https://t.co/CJ02UHvxVY"}

## Análise de Dados com Pandas e Scikit-Learn

In [20]:
# criando um dataset com dados retornados do MongoDB
#Retornar os dados para cada item na coleção (Cada item é um documento)
dataset = [{"created_at": item["created_at"], "text": item["text"],} for item in col.find()]

In [21]:
# Importando o módulo Pandas para trabalhar com datasets em Python
import pandas as pd

In [22]:
# Criando um dataframe a partir do dataset 
df = pd.DataFrame(dataset) #Criação de um dataframe (objetos com formato de colunas e linhas) a partir do banco de dados dataset presente no mongodb

In [23]:
# Imprimindo o dataframe
df

,created_at,text
0,Sat Nov 30 14:06:05 +0000 2019,RT @Neenar568: OK...it's a toss up~~I'm dying!...
1,Sat Nov 30 14:06:05 +0000 2019,@AdeBARRcode @ravenhaired_sub In essence it is...
2,Sat Nov 30 14:06:05 +0000 2019,RT @jessphillips: Answer the question you spin...
3,Sat Nov 30 14:06:05 +0000 2019,Pahiram netflix acc HAHA
4,Sat Nov 30 14:06:06 +0000 2019,"""our Netflix"" sounds so domestic can you."
5,Sat Nov 30 14:06:06 +0000 2019,SAME! Definitely nearing triple digits #renewa...
6,Sat Nov 30 14:06:06 +0000 2019,"My fucking 123movuee geekin , lemme hold someb..."
7,Sat Nov 30 14:06:07 +0000 2019,RT @travelkaan: น่ารักมาก ที่พักที่เวียดนาม ถู...
8,Sat Nov 30 14:06:07 +0000 2019,Ultimato😱
9,Sat Nov 30 14:06:07 +0000 2019,@Mr_Barnett10 8 hours of netflix...


In [24]:
# Importando o módulo Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
# Usando o método CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [26]:
# Contando o número de ocorrências das principais palavras em nosso dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]

,word,count
0,rt,40
1,netflix,29
2,https,23
3,co,22
4,the,17
5,and,11
6,is,9
7,to,9
8,it,8
9,we,7


# Fim

### Obrigado - Data Science Academy - <a href="http://facebook.com/dsacademybr">facebook.com/dsacademybr</a>